In [1]:
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import ast

def parse_scores_from_file(file_path):
    scores = {}
    with open(file_path, 'r') as file:
        data = file.read()

        # Corpus BLEU scores
        bleu_scores = re.findall(r'corpus BLEU-(\d) score: (\d+\.\d+)', data)
        for n, score in bleu_scores:
            scores[f'BLEU-{n}'] = float(score)

        # ROUGE scores
        rouge_scores_match = re.search(r'ROUGE score: ({.+})', data)
        if rouge_scores_match:
            rouge_scores_str = rouge_scores_match.group(1)
            rouge_scores = ast.literal_eval(rouge_scores_str)
            for rouge_type, rouge_values in rouge_scores.items():
                for metric, value in rouge_values.items():
                    scores[f'{rouge_type}-{metric}'] = value

    return scores

In [5]:
path = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\EEG-To-Text\best\Contextual"
dir_list = os.listdir(path)

In [6]:
#reading in files and assigning categories
list_of_dfs = []
for file in dir_list:
    if "25" in file:
        augmentation_size = "25"
    elif "100" in file:
        augmentation_size = "100"
    elif "50" in file:
        augmentation_size = "50"
    elif "75" in file:
        augmentation_size = "75"
    elif "40" in file:
        augmentation_size = "40"
    elif "60" in file:
        augmentation_size = "60"
    elif "55" in file:
        augmentation_size = "55"

    if "WGAN_v1_Text" in file:
        model = "WGAN_v1_Text"
    elif "DCGAN_v1_Text" in file:
        model = "DCGAN_v1_Text"
    elif "WGAN_v2_Text" in file:
        model = "WGAN_v2_Text"
    elif "DCGAN_v2_Text" in file:
        model = "DCGAN_v2_Text"

    #augmentation type
    if "random" in file:
        augmentation_type = "random"
    elif "TF-IDF-Low" in file:
        augmentation_type = "TF-IDF-Low"
    elif "TF-IDF-High" in file:
        augmentation_type = "TF-IDF-High"
    elif "TF-IDF-Medium" in file:
        augmentation_type = "TF-IDF-Medium"

    #generation type:
    if "Word_Level" in file:
        generation_type = "Word_Level"
    elif "Sentence_Level" in file:
        generation_type = "Sentence_Level"
    elif "Contextual" in file:
        generation_type = "Contextual"

    scores = parse_scores_from_file(rf"{path}\{file}")
    df = pd.DataFrame([scores])
    df["augmentation_size"] = augmentation_size
    df["model"] = model
    df["augmentation_type"] = augmentation_type
    df["generation_type"] = generation_type

    list_of_dfs.append(df)

In [38]:
df = pd.concat(list_of_dfs)

In [40]:
df

,BLEU-1,BLEU-2,BLEU-3,BLEU-4,rouge-1-r,rouge-1-p,rouge-1-f,rouge-2-r,rouge-2-p,rouge-2-f,rouge-l-r,rouge-l-p,rouge-l-f,augmentation_size,model,augmentation_type,generation_type
0,0.109529,0.038105,0.019852,0.004138,0.121724,0.122846,0.098051,0.025548,0.031165,0.026433,0.126657,0.105167,0.112517,100,DCGAN_v2_Text,random,Contextual
0,0.110153,0.038105,0.019852,0.004138,0.121724,0.122846,0.099051,0.025548,0.031165,0.026433,0.126657,0.105167,0.112517,100,DCGAN_v2_Text,random,Sentence_Level
0,0.099529,0.038105,0.019852,0.004138,0.121724,0.122846,0.091051,0.025548,0.031165,0.026433,0.126657,0.105167,0.112517,100,DCGAN_v2_Text,random,Word_Level
0,0.128546,0.026589,0.012785,0.002838,0.114272,0.141457,0.121494,0.013555,0.021165,0.015643,0.105666,0.147517,0.122517,100,DCGAN_v2_Text,random,Contextual
0,0.135458,0.028888,0.017853,0.002380,0.122724,0.131457,0.141494,0.013555,0.021654,0.016433,0.125666,0.157517,0.123325,100,DCGAN_v2_Text,random,Sentence_Level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.145929,0.048105,0.020852,0.005138,0.121724,0.144228,0.132251,0.022555,0.031165,0.022643,0.215666,0.147517,0.152517,60,DCGAN_v2_Text,random,Sentence_Level
0,0.141529,0.048105,0.020852,0.005138,0.121724,0.144228,0.130121,0.022555,0.031165,0.022643,0.215666,0.147517,0.152517,60,DCGAN_v2_Text,random,Word_Level
0,0.121529,0.038105,0.019852,0.004138,0.121724,0.122846,0.124051,0.025548,0.031165,0.026433,0.126657,0.105167,0.112517,75,DCGAN_v2_Text,random,Contextual
0,0.137012,0.018105,0.016852,0.004238,0.121724,0.102846,0.139051,0.022548,0.031654,0.023433,0.126657,0.105167,0.112517,75,DCGAN_v2_Text,random,Sentence_Level


In [42]:
#average accuracy for each augmentation size with standard deviation
#just BLEU-1 and rouge-1-f and standard deviation for contextual only
contextual = df[df["generation_type"] == "Contextual"]
contextual = contextual[["BLEU-1", "rouge-1-f", "augmentation_size"]]
contextual = contextual.groupby("augmentation_type").agg(["mean", "std"])

#only 25, 55, 75, and 100
contextual = contextual.loc[["25", "55", "75", "100"]]

word_level = df[df["generation_type"] == "Word_Level"]
word_level = word_level[["BLEU-1", "rouge-1-f", "augmentation_size"]]
word_level = word_level.groupby("augmentation_size").agg(["mean", "std"])

word_level = word_level.loc[["25", "55", "75", "100"]]

sentence_level = df[df["generation_type"] == "Sentence_Level"]
sentence_level = sentence_level[["BLEU-1", "rouge-1-f", "augmentation_size"]]
sentence_level = sentence_level.groupby("augmentation_size").agg(["mean", "std"])

sentence_level = sentence_level.loc[["25", "55", "75", "100"]]



KeyError: "None of [Index(['25', '55', '75', '100'], dtype='object', name='augmentation_type')] are in the [index]"

In [32]:
contextual

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_size                                        
25                 0.120450  0.041639  0.118965  0.025753
55                 0.140294       NaN  0.136051       NaN
75                 0.121529       NaN  0.124051       NaN
100                0.119086  0.019711  0.118868  0.019129

In [33]:
sentence_level

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_size                                        
25                 0.132441  0.022749  0.118836  0.022132
55                 0.165029       NaN  0.152001       NaN
75                 0.137012       NaN  0.139051       NaN
100                0.115428  0.016566  0.122503  0.015207

In [34]:
word_level

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_size                                        
25                 0.129482  0.025163  0.124639  0.014464
55                 0.146294       NaN  0.135051       NaN
75                 0.110529       NaN  0.120051       NaN
100                0.115494  0.013927  0.113215  0.018192

In [44]:
#average accuracy for each augmentation type with standard deviation
#just BLEU-1 and rouge-1-f and standard deviation for contextual only
contextual = df[df["generation_type"] == "Contextual"]
contextual = contextual[["BLEU-1", "rouge-1-f", "augmentation_type"]]
contextual = contextual.groupby("augmentation_type").agg(["mean", "std"])

In [45]:

word_level = df[df["generation_type"] == "Word_Level"]
word_level = word_level[["BLEU-1", "rouge-1-f", "augmentation_type"]]
word_level = word_level.groupby("augmentation_type").agg(["mean", "std"])

sentence_level = df[df["generation_type"] == "Sentence_Level"]
sentence_level = sentence_level[["BLEU-1", "rouge-1-f", "augmentation_type"]]
sentence_level = sentence_level.groupby("augmentation_type").agg(["mean", "std"])


In [46]:
contextual

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_type                                        
TF-IDF-High        0.118289  0.024743  0.128824  0.005424
TF-IDF-Low         0.129806  0.026946  0.114126  0.017663
TF-IDF-Medium      0.145783  0.014791  0.121691  0.010243
random             0.125540  0.036294  0.124339  0.023521

In [47]:
sentence_level

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_type                                        
TF-IDF-High        0.120560  0.023120  0.125630  0.022495
TF-IDF-Low         0.138135  0.029657  0.113659  0.015691
TF-IDF-Medium      0.138298  0.015573  0.116912  0.013736
random             0.132361  0.022973  0.133307  0.016037

In [48]:
word_level

BLEU-1           rouge-1-f          
                       mean       std      mean       std
augmentation_type                                        
TF-IDF-High        0.114442  0.033008  0.120874  0.024323
TF-IDF-Low         0.120305  0.017727  0.121475  0.015149
TF-IDF-Medium      0.118587  0.017385  0.120375  0.015289
random             0.130987  0.014569  0.123141  0.011352